In [ ]:
import warnings
import numpy as np
from unidecode import unidecode
import pandas as pd
warnings.filterwarnings('ignore')
class DataProcessing:
    def __init__(self, filename: str) -> None:
        if filename.endswith(".xlsx"):
            self.df = pd.read_excel(filename, skiprows=8, skipfooter=2, header=[0])
        if filename.endswith(".csv"):
            self.df = pd.read_csv(filename, skiprows=8, skipfooter=2, header=[0])
        self.df.replace(np.nan, -1, inplace=True)
        self.df['Mã sinh viên'] = self.df['Mã sinh viên'].astype(int).astype(str)
        
    def get_subject_credit(self):
        df = self.df.drop(columns=['STT', 'Mã sinh viên', 'Họ đệm', 'Tên', 'Học lực'], axis=0)
        df.drop(columns=df.columns[-6:], axis=0, inplace=True)
        return df.iloc[0].to_dict()
    
    def get_all_student_detail(self):
        first_col = self.df.columns[1]
        last_col = self.df.columns[-7:-2]
        df = self.df[[first_col] + list(last_col)]
        df.columns = df.iloc[0]
        df.drop(index=0, inplace=True)
        df.columns = ['student_id', 'passed_credit', 'score_10', 'score_4', 'score_char', 'rank']
        res = df.to_dict(orient='records')
        
        df = self.df[['Mã sinh viên', 'Họ đệm', 'Tên']].drop(index=0)
        df.columns = ['student_id', 'first_name', 'last_name']
        df['student_name'] = df['first_name'] + " " + df['last_name']
        df['last_name'] = df.apply(lambda x: unidecode(x['last_name'].lower()), axis=1)
        df['student_gmail'] = df.apply(lambda x: f"{x['last_name']}.{x['student_id']}@iuh.edu.vn", axis=1)
        df.drop(columns=['first_name', 'last_name'], inplace=True)
        data = df.to_dict(orient='records')
        
        dict1_map = {d['student_id']: d for d in data}
        res = [dict({**d, **dict1_map.get(d['student_id'], {})}) for d in res]
        
        return res